In [2]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-238817
Azure region: southcentralus
Subscription id: 510b94ba-e453-4417-988b-fbdc37b55ca7
Resource group: aml-quickstarts-238817


In [13]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = "Udacity-Project-Cluster2"
compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                              max_nodes=4);
cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)



InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [21]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

ps = RandomParameterSampling( {
        "first-layer-neurons" : choice(25, 50, 200,300, 500),
        "second-layer-neurons" : choice(25, 50, 200,300, 500),
        "learning_rate": uniform(0.5, 1.0),
        "batch_size": choice(25, 50, 100)
    })

policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

src=ScriptRunConfig(source_directory="./",  script="train.py", arguments="--C 1.0 --max_iter 100", run_config=None, _telemetry_values=None, compute_target=cpu_cluster, environment=sklearn_env, distributed_job_config=None, resume_from=None, max_run_duration_seconds=2592000, command=None, docker_runtime_config=None)

hyperdrive_config  = HyperDriveConfig(hyperparameter_sampling = ps , primary_metric_name = "Accuracy", primary_metric_goal = PrimaryMetricGoal.MAXIMIZE, max_total_runs=20, max_concurrent_runs=2, max_duration_minutes=10080, policy=policy, estimator=None, run_config=src, resume_from=None, resume_child_runs=None, pipeline=None, debug_flag=None, custom_run_id=None)

hdr= exp.submit(config=hyperdrive_config)

RunDetails(hdr).show()


TrainingException: TrainingException:
	Message: Exception occurred while creating the HyperDrive run: [Operation returned an invalid status code 'udacity-project Error : User Error occured while deserializing input, JsonReaderException was thrown']. Please try again later.
	InnerException Operation returned an invalid status code 'udacity-project Error : User Error occured while deserializing input, JsonReaderException was thrown'
	ErrorResponse 
{
    "error": {
        "code": "SystemError",
        "message": "Exception occurred while creating the HyperDrive run: [Operation returned an invalid status code 'udacity-project Error : User Error occured while deserializing input, JsonReaderException was thrown']. Please try again later.",
        "inner_error": {
            "code": "ClientError",
            "inner_error": {
                "code": "HyperDriveTrainingError"
            }
        }
    }
}